## This project focuses on implementing machine learning in credit risk predictions. Detail description will be updated shortly.

# PD model

## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Import Data

In [ ]:
loan_data_backup = pd.read_csv('loan_data.csv')

In [ ]:
loan_data = loan_data_backup.copy()

In [ ]:
pd.options.display.max_columns = None

In [ ]:
loan_data.head()

In [ ]:
loan_data.columns.values

In [ ]:
loan_data.info()

# General Processing

## Preprocessing few continuous variables

### Turn 'emp_length' to float

In [ ]:
loan_data['emp_length'].unique()

In [ ]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a', str(0))

In [ ]:
loan_data['emp_length_int'].head(10)

In [ ]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])

In [ ]:
type(loan_data['emp_length_int'][0])

In [ ]:
loan_data['emp_length_int'].unique()

In [ ]:
loan_data['emp_length_int'].isnull().sum()

### Turn 'term' into int

In [ ]:
loan_data['term'].unique()

In [ ]:
loan_data['term_int'] = loan_data['term'].str.split(expand=True)[0]

In [ ]:
loan_data['term_int'] = pd.to_numeric(loan_data['term_int'])

In [ ]:
loan_data['term_int'].unique()

### Turn 'earliest_cr_line' and '' into months since

In [ ]:
loan_data['earliest_cr_line'].head()

In [ ]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format='%b-%y')

In [ ]:
loan_data['earliest_cr_line_date'].head()

In [ ]:
# date = pd.datetime.today()
date = pd.to_datetime('2017-12-01')

In [ ]:
date - loan_data['earliest_cr_line_date'].head()

In [ ]:
loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((date - loan_data['earliest_cr_line_date']) / np.timedelta64(1,'M')))

In [ ]:
loan_data['mths_since_earliest_cr_line'].describe()

In [ ]:
loan_data.loc[:,['earliest_cr_line','earliest_cr_line_date','mths_since_earliest_cr_line']][loan_data['mths_since_earliest_cr_line']<0]

In [ ]:
loan_data['mths_since_earliest_cr_line'][loan_data['mths_since_earliest_cr_line']<0] = loan_data['mths_since_earliest_cr_line'].max() 

In [ ]:
min(loan_data['mths_since_earliest_cr_line'])

### Turn 'issue_d' and '' into timestamp

In this homework, your task is to preprocess the ‘term’ variable and the ‘issue_d’ variable, similarly to the ‘employment length’ variable and the ‘earliest credit line variable’ we preprocessed in the last lecture. 1. From the variable ‘term’, which is of object (text string) data type, create a new variable, named ‘term_int’, which is numeric and contains the term of the loan in months. Tips: First remove the unnecessary text from each row; then, turn what remains into a numeric variable. 2. From the variable ‘issue date’ (‘issue_d’), which is of object (text string) data type, create a new variable, named ‘mths_since_issue_d’ (‘months since issue date’). Tips: First turn the test strings into dates; then calculate the difference between December 1st 2017 and each of the issue dates, in months; finally, check if the values make sense.

In [ ]:
loan_data['issue_d_date'] = pd.to_datetime(loan_data['issue_d'], format='%b-%y')

In [ ]:
date - pd.to_datetime(loan_data['issue_d'], format='%b-%y')

In [ ]:
loan_data['mths_since_issue_d'] = round(pd.to_numeric((date - loan_data['issue_d_date']) / np.timedelta64(1,'M')))

In [ ]:
loan_data['mths_since_issue_d'].unique()

In [ ]:
loan_data['mths_since_issue_d'].describe()

### Dummy discrete variables

In [ ]:
loan_data.columns.values

In [ ]:
loan_data['grade'].head()

In [ ]:
loan_data_dummies = pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep=':')

In [ ]:
loan_data_dummies = [
    pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep=':'),
    pd.get_dummies(loan_data['sub_grade'], prefix = 'sub_grade', prefix_sep=':'),
    pd.get_dummies(loan_data['home_ownership'], prefix = 'home_ownership', prefix_sep=':'),
    pd.get_dummies(loan_data['verification_status'], prefix = 'verification_status', prefix_sep=':'),
    pd.get_dummies(loan_data['loan_status'], prefix = 'loan_status', prefix_sep=':'),
    pd.get_dummies(loan_data['purpose'], prefix = 'purpose', prefix_sep=':'),
    pd.get_dummies(loan_data['addr_state'], prefix = 'addr_state', prefix_sep=':'),
    pd.get_dummies(loan_data['initial_list_status'], prefix = 'initial_list_status', prefix_sep=':')
]

In [ ]:
loan_data_dummies = pd.concat(loan_data_dummies, axis=1)

In [ ]:
type(loan_data_dummies)

In [ ]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis=1)

In [ ]:
loan_data.columns.values

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
loan_data.isnull().sum().values

In [ ]:
loan_data.isnull().sum() /loan_data.shape[0]

In [ ]:
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace = True)

In [ ]:
loan_data['total_rev_hi_lim'].isnull().sum()

In this homework, your task is to fill in the missing values for the remaining variables we are going to use in our models. Here are the instructions about what values to replace the missing values with for the remaining variables. 1. For annual income (‘annual_inc’), replace the missing values with the mean of the non-missing values of the variable. 2. For all of the following variables, replace the missing values with zeroes: ‘mths_since_earliest_cr_line’ ‘acc_now_delinq’ ‘total_acc’ ‘pub_rec’ ‘open_acc’ ‘inq_last_6mths’ ‘delinq_2yrs’ ‘emp_length_int’

In [ ]:
loan_data['annual_inc'].isnull().sum()

In [ ]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(), inplace=True)

In [ ]:
loan_data['annual_inc'].isnull().sum()

In [ ]:
loan_data.update(loan_data[['mths_since_earliest_cr_line', 'acc_now_delinq', 'total_acc','pub_rec','open_acc','inq_last_6mths','delinq_2yrs','emp_length_int']].fillna(0))

In [ ]:
loan_data[['mths_since_earliest_cr_line', 'acc_now_delinq', 'total_acc','pub_rec','open_acc','inq_last_6mths','delinq_2yrs','emp_length_int']].isnull().sum()

create good/bad definition

In [ ]:
loan_data['loan_status'].unique()

In [ ]:
loan_data['loan_status'].value_counts()/loan_data['loan_status'].count()

In [ ]:
loan_data['good_bad'] =np.where(loan_data['loan_status'].isin(['Charged Off','Default', 'Does not meet the credit policy. Status:Charged Off', 'Late (31-120 days)']),0,1)

In [ ]:
loan_data['good_bad'].unique()

Independent Variables into dummies according to **WoE**

fine classing -> WoE calculation -> Coarse classing

# $\text{WoE}_{i}=\ln \left(\frac{\%(y=1)_i}{\%(y=0)_i}\right)$

Information Value (IF)

# $\text{IF} = \sum_{i=1}^{k}\left[\left(\%(y=1)_i-\%(y=0)_i\right)\right]\times\text{WoE_i}$
Suppose there are k categories for this variable. Either continuous after classing or discrete.

More advanced methods: classing with WoE and sample size. abnormal values goes to the two side classes.

# splitting data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(loan_data.drop('good_bad',axis=1), loan_data['good_bad'], test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
#df_inputs_pre = X_train
#df_targets_pre = y_train
df_inputs_pre = X_test
df_targets_pre = y_test

In [ ]:
df_inputs_pre['grade'].unique()

In [ ]:
df1 = pd.concat([df_inputs_pre['grade'], df_targets_pre],axis=1)
df1.head()

In [ ]:
df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].count()

In [ ]:
df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].mean()

In [ ]:
df1 = pd.concat([df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].count(),
                df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].mean()], axis=1)

In [ ]:
df1.head()

In [ ]:
df1 = df1.iloc[:,[0,1,3]]
df1.columns = [df1.columns.values[0], 'n_obs', 'prop_good']

In [ ]:
df1['prop_n_obs'] = df1['n_obs']/df1['n_obs'].sum()

In [ ]:
df1['n_good'] = df1['prop_good'] * df1['n_obs']
df1['n_bad'] = (1 - df1['prop_good']) * df1['n_obs']

In [ ]:
df1['prop_n_good'] = df1['n_good'] / df1['n_good'].sum()
df1['prop_n_bad'] = df1['n_bad'] / df1['n_bad'].sum()

In [ ]:
df1['WoE'] = np.log(df1['prop_n_good']/df1['prop_n_bad'])

In [ ]:
df1 = df1.sort_values(['WoE'])
df1 = df1.reset_index(drop = True)

In [ ]:
df1['diff_prop_good'] = df1['prop_good'].diff().abs()
df1['diff_WoE'] = df1['WoE'].diff().abs()

In [ ]:
df1['IV'] = (df1['prop_n_good']-df1['prop_n_bad'])*df1['WoE']
df1['IV'] = df1['IV'].sum()

In [ ]:
df1

# Automatic WoE and IV calculator

In [ ]:
def woe_discrete(df, discrete_variable_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variable_name], good_bad_variable_df],axis=1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index=False)[df.columns.values[1]].count(),
                df.groupby(df.columns.values[0], as_index=False)[df.columns.values[1]].mean()], axis=1)
    df = df.iloc[:,[0,1,3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs']/df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good']/df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good']-df['prop_n_bad'])*df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'grade', df_targets_pre)

In [ ]:
df_temp

# WoE Plotter

In [ ]:
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.iloc[:, 0].apply(str))
    y = df_WoE['WoE']
    plt.figure(figsize = (18, 6))
    plt.plot(x, y, marker='o', linestyle='--', color='k')
    plt.xlabel(df_WoE.columns[0])
    plt.ylabel('Weight of Evidence')
    plt.title(str('Weight of Evidence ' + df_WoE.columns[0]))
    plt.xticks(rotation = rotation_of_x_axis_labels)

In [ ]:
plot_by_woe(df_temp)

## Do rest discrete variables

'home_ownership'

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'home_ownership', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp)

Due to high risk and low number of samples, we combine the 'ANY', 'OTHER' and 'NONE' categories to the riskest category with enough number of observations, i.e. 'RENT'

In [ ]:
df_inputs_pre['home_ownership:RENT_OTHER_NONE_ANY'] = sum([df_inputs_pre['home_ownership:RENT'], df_inputs_pre['home_ownership:OTHER'], df_inputs_pre['home_ownership:NONE'], df_inputs_pre['home_ownership:ANY']])

'addr_state'

In [ ]:
df_inputs_pre['addr_state'].unique()

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'addr_state', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
if ['addr_state:ND'] in df_inputs_pre.columns.values:
    pass
else:
    df_inputs_pre['addr_state:ND'] = 0

In [ ]:
plot_by_woe(df_temp[2:-2])

In [ ]:
# combine ND, NE, IA, NV, FL, HI, and AL
df_inputs_pre['addr_state:ND_NE_IA_NV_FL_HI_AL'] = sum([df_inputs_pre['addr_state:ND'], df_inputs_pre['addr_state:NE'], df_inputs_pre['addr_state:IA'], df_inputs_pre['addr_state:NV'], df_inputs_pre['addr_state:FL'], df_inputs_pre['addr_state:HI'], df_inputs_pre['addr_state:AL']])

In [ ]:
# combine WV, NH, WY, DC, ME, ID
df_inputs_pre['addr_state:WV_NH_WY_DC_ME_ID'] = sum([df_inputs_pre['addr_state:WV'], df_inputs_pre['addr_state:NH'], df_inputs_pre['addr_state:WY'], df_inputs_pre['addr_state:DC'], df_inputs_pre['addr_state:ME'], df_inputs_pre['addr_state:ID']])

In [ ]:
plot_by_woe(df_temp[6:-6])

In [ ]:
# combine NM, VA, keep NY
df_inputs_pre['addr_state:NM_VA'] = sum([df_inputs_pre['addr_state:NM'], df_inputs_pre['addr_state:VA']])

In [ ]:
# combine OK, TN, MO, LA, MD, NC, keep CA
df_inputs_pre['addr_state:OK_TN_MO_LA_MD_NC'] = sum([df_inputs_pre['addr_state:OK'], df_inputs_pre['addr_state:TN'], df_inputs_pre['addr_state:MO'], df_inputs_pre['addr_state:LA'], df_inputs_pre['addr_state:MD'], df_inputs_pre['addr_state:NC']])

In [ ]:
# combine UT, KY, AZ, NJ
df_inputs_pre['addr_state:UT_KY_AZ_NJ'] = sum([df_inputs_pre['addr_state:UT'], df_inputs_pre['addr_state:KY'], df_inputs_pre['addr_state:AZ'], df_inputs_pre['addr_state:NJ']])

In [ ]:
# combine AR, MI, PA, OH, MN
df_inputs_pre['addr_state:AR_MI_PA_OH_MN'] = sum([df_inputs_pre['addr_state:AR'], df_inputs_pre['addr_state:MI'], df_inputs_pre['addr_state:PA'], df_inputs_pre['addr_state:OH'], df_inputs_pre['addr_state:MN']])

In [ ]:
# combine RI, MA, DE, SD, IN
df_inputs_pre['addr_state:RI_MA_DE_SD_IN'] = sum([df_inputs_pre['addr_state:RI'], df_inputs_pre['addr_state:MA'], df_inputs_pre['addr_state:DE'], df_inputs_pre['addr_state:SD'], df_inputs_pre['addr_state:IN']])

In [ ]:
# combine GA, WA, OR
df_inputs_pre['addr_state:GA_WA_OR'] = sum([df_inputs_pre['addr_state:GA'], df_inputs_pre['addr_state:WA'], df_inputs_pre['addr_state:OR']])

In [ ]:
# combine WI, MT, keep TX
df_inputs_pre['addr_state:WI_MT'] = sum([df_inputs_pre['addr_state:WI'], df_inputs_pre['addr_state:MT']])

In [ ]:
# combine IL, CT
df_inputs_pre['addr_state:IL_CT'] = sum([df_inputs_pre['addr_state:IL'], df_inputs_pre['addr_state:CT']])

In [ ]:
# combine KS, SC, CO, VT, AK, MS
df_inputs_pre['addr_state:KS_SC_CO_VT_AK_MS'] = sum([df_inputs_pre['addr_state:KS'], df_inputs_pre['addr_state:CO'], df_inputs_pre['addr_state:SC'], df_inputs_pre['addr_state:VT'], df_inputs_pre['addr_state:AK'], df_inputs_pre['addr_state:MS']])

'verification_status'

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'verification_status', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'verification_status:Not Verified', 'verification_status:Source Verified', verification_status:Verified

'purpose'

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'purpose', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
# 'purpose:smabus_edu_mov_hou_oth', 'purpose:reen_med_wed_vac', 'purpose:debt_consolidation', 'purpose:hom_maj_car', 'purpose:credit_card'
#df_inputs_pre['purpose:']
df_inputs_pre['purpose:smabus_edu_mov_hou_oth'] = sum([df_inputs_pre['purpose:small_business'], df_inputs_pre['purpose:educational'], df_inputs_pre['purpose:moving'], df_inputs_pre['purpose:house'], df_inputs_pre['purpose:other']])

df_inputs_pre['purpose:reen_med_wed_vac'] = sum([df_inputs_pre['purpose:renewable_energy'], df_inputs_pre['purpose:medical'], df_inputs_pre['purpose:wedding'], df_inputs_pre['purpose:vacation']])

df_inputs_pre['purpose:hom_maj_car'] = sum([df_inputs_pre['purpose:home_improvement'], df_inputs_pre['purpose:major_purchase'], df_inputs_pre['purpose:car']])

'initial_list_status'

In [ ]:
df_temp = woe_discrete(df_inputs_pre, 'initial_list_status', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
#'initial_list_status:f', 'initial_list_status:w'

### Continuous varialbes processing

In [ ]:
def woe_ordered_continuous(df, continuous_variable_name, good_bad_variable_df):
    df = pd.concat([df[continuous_variable_name], good_bad_variable_df],axis=1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index=False)[df.columns.values[1]].count(),
    df.groupby(df.columns.values[0], as_index=False)[df.columns.values[1]].mean()], axis=1)
    df = df.iloc[:,[0,1,3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs']/df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good']/df['prop_n_bad'])
    # df = df.sort_values(['WoE'])
    # df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good']-df['prop_n_bad'])*df['WoE']
    df['IV'] = df['IV'].sum()
    return df

'term_int' variable

In [ ]:
df_inputs_pre['term_int'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'term_int', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'term:36', 'term:60'
df_inputs_pre['term:36'] = np.where(df_inputs_pre['term_int'] == 36, 1, 0)
df_inputs_pre['term:60'] = np.where(df_inputs_pre['term_int'] == 60, 1, 0)

'emp_length_int'

In [ ]:
df_inputs_pre['emp_length_int'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'emp_length_int', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# add 'emp_length:0', add 'emp_length:1', add 'emp_length:2-4', add 'emp_length:5-6', add 'emp_length:7-9', add 'emp_length:10'
df_inputs_pre['emp_length:0'] = np.where(df_inputs_pre['emp_length_int'].isin([0]),1,0)
df_inputs_pre['emp_length:1'] = np.where(df_inputs_pre['emp_length_int'].isin([1]),1,0)
df_inputs_pre['emp_length:2-4'] = np.where(df_inputs_pre['emp_length_int'].isin(range(2,5)),1,0)
df_inputs_pre['emp_length:5-6'] = np.where(df_inputs_pre['emp_length_int'].isin(range(5,7)),1,0)
df_inputs_pre['emp_length:7-9'] = np.where(df_inputs_pre['emp_length_int'].isin(range(7,10)),1,0)
df_inputs_pre['emp_length:10'] = np.where(df_inputs_pre['emp_length_int'].isin([10]),1,0)

'mths_since_issue_d'

In [ ]:
df_inputs_pre['mths_since_issue_d'].unique()

In [ ]:
df_inputs_pre['mths_since_issue_d_factor'] = pd.cut(df_inputs_pre['mths_since_issue_d'],50)
df_inputs_pre['mths_since_issue_d_factor']

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'mths_since_issue_d_factor', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
plot_by_woe(df_temp.iloc[3:,:], 90)

In [ ]:
# 'mths_since_issue_d::<38', 'mths_since_issue_d:38-39', 'mths_since_issue_d:40-41', 'mths_since_issue_d:42-48', mths_since_issue_d:49-52', 'mths_since_issue_d:53-64', 'mths_since_issue_d:65-84', 'mths_since_issue_d:>84'
df_inputs_pre['mths_since_issue_d::<38'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(38)),1,0)
df_inputs_pre['mths_since_issue_d:38-39'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(38,40)),1,0)  
df_inputs_pre['mths_since_issue_d:40-41'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(40,42)),1,0)  
df_inputs_pre['mths_since_issue_d:42-48'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(42,49)),1,0)
df_inputs_pre['mths_since_issue_d:49-52'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(49,53)),1,0)  
df_inputs_pre['mths_since_issue_d:53-64'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(53,65)),1,0)  
df_inputs_pre['mths_since_issue_d:65-84'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(65,85)),1,0)  
df_inputs_pre['mths_since_issue_d:>84'] = np.where(df_inputs_pre['mths_since_issue_d'].isin(range(85, int(df_inputs_pre['mths_since_issue_d'].max()))),1,0)  

'int_rate'

In [ ]:
df_inputs_pre['int_rate_factor'] = pd.cut(df_inputs_pre['int_rate'],50)

In [ ]:
df_inputs_pre['int_rate_factor']

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'int_rate_factor', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'int_rate:<9.548', 'int_rate:9.548-12.025', 'int_rate:12.025-15.74', 'int_rate:15.75-20.281', 'int_rate:>20.281'
df_inputs_pre['int_rate:<9.548'] = np.where((df_inputs_pre['int_rate']< 9.548),1,0)
df_inputs_pre['int_rate:9.548-12.025'] = np.where((df_inputs_pre['int_rate'] >= 9.548 ) & (df_inputs_pre['int_rate'] < 12.025), 1, 0)
df_inputs_pre['int_rate:12.025-15.74'] = np.where((df_inputs_pre['int_rate'] >= 12.025) & (df_inputs_pre['int_rate'] < 15.74), 1, 0)
df_inputs_pre['int_rate:15.75-20.281'] = np.where((df_inputs_pre['int_rate'] >= 15.74) & (df_inputs_pre['int_rate'] < 20.281), 1, 0)
df_inputs_pre['int_rate:>20.281'] = np.where((df_inputs_pre['int_rate'] >= 20.281), 1, 0)

'funded_amnt'

In [ ]:
df_inputs_pre['funded_amnt_factor'] = pd.cut(df_inputs_pre['funded_amnt'], 50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'funded_amnt_factor', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp,90)

not to use 'funded_amnt'

'mths_since_earliest_cr_line'

In [ ]:
df_inputs_pre['mths_since_earliest_cr_line_factor'] = pd.cut(df_inputs_pre['mths_since_earliest_cr_line'],50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'mths_since_earliest_cr_line_factor', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
plot_by_woe(df_temp.iloc[6:,:],90)

In [ ]:
# 'mths_since_earliest_cr_line:<140', 'mths_since_earliest_cr_line:141-164', 'mths_since_earliest_cr_line:165-247', 'mths_since_earliest_cr_line:248-270', 'mths_since_earliest_cr_line:271-352', 'mths_since_earliest_cr_line:>352'

df_inputs_pre['mths_since_earliest_cr_line:<140'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(140)),1,0)

df_inputs_pre['mths_since_earliest_cr_line:141-164'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(141,165)),1,0)

df_inputs_pre['mths_since_earliest_cr_line:165-247'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(165,248)),1,0)

df_inputs_pre['mths_since_earliest_cr_line:248-270'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(248,271)),1,0)

df_inputs_pre['mths_since_earliest_cr_line:271-352'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(271,353)),1,0)

df_inputs_pre['mths_since_earliest_cr_line:>352'] = np.where(df_inputs_pre['mths_since_earliest_cr_line'].isin(range(353)),1,0)

'delinq_2yrs'

In [ ]:
df_inputs_pre['delinq_2yrs'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'delinq_2yrs', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'delinq_2yrs:0', 'delinq_2yrs:1-3', 'delinq_2yrs:>=4'
df_inputs_pre['delinq_2yrs:0'] = np.where((df_inputs_pre['delinq_2yrs'] == 0), 1, 0)

df_inputs_pre['delinq_2yrs:1-3'] = np.where((df_inputs_pre['delinq_2yrs'] >=1) & (df_inputs_pre['delinq_2yrs'] <=3), 1, 0)

df_inputs_pre['delinq_2yrs:>=4'] = np.where((df_inputs_pre['delinq_2yrs'] >=4), 1, 0)

'inq_last_6mths'

In [ ]:
df_inputs_pre['inq_last_6mths'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'inq_last_6mths', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'inq_last_6mths:0', 'inq_last_6mths:1-2', 'inq_last_6mths:3-6', 'inq_last_6mths:>6'

df_inputs_pre['inq_last_6mths:0'] = np.where((df_inputs_pre['inq_last_6mths'] == 0),  1, 0)
df_inputs_pre['inq_last_6mths:1-2'] = np.where((df_inputs_pre['inq_last_6mths'] >=1) & (df_inputs_pre['inq_last_6mths']<=2),  1, 0)
df_inputs_pre['inq_last_6mths:3-6'] = np.where((df_inputs_pre['inq_last_6mths']>=3) & (df_inputs_pre['inq_last_6mths']<=6),  1, 0)
df_inputs_pre['inq_last_6mths:>6'] = np.where((df_inputs_pre['inq_last_6mths']>6),  1, 0)

'open_acc'

In [ ]:
df_inputs_pre['open_acc'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'open_acc', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
plot_by_woe(df_temp.iloc[:40,:],90)

In [ ]:
# 'open_acc:0', 'open_acc:1-3', 'open_acc:4-12', 'open_acc:13-22', 'open_acc:23-25', 'open_acc:26-30' 'open_acc:>30'

df_inputs_pre['open_acc:0'] = np.where((df_inputs_pre['open_acc'] ==0), 1, 0)

df_inputs_pre['open_acc:1-3'] = np.where((df_inputs_pre['open_acc'] >= 1) & (df_inputs_pre['open_acc'] <= 3), 1, 0)

df_inputs_pre['open_acc:4-12'] = np.where((df_inputs_pre['open_acc'] >= 4) & (df_inputs_pre['open_acc'] <= 12), 1, 0)

df_inputs_pre['open_acc:13-22'] = np.where((df_inputs_pre['open_acc'] >= 13) & (df_inputs_pre['open_acc'] <= 22), 1, 0)

df_inputs_pre['open_acc:23-25'] = np.where((df_inputs_pre['open_acc'] >= 23) & (df_inputs_pre['open_acc'] <= 25), 1, 0)

df_inputs_pre['open_acc:26-30'] = np.where((df_inputs_pre['open_acc'] >= 26) & (df_inputs_pre['open_acc'] <= 30), 1, 0)

df_inputs_pre['open_acc:>30'] = np.where((df_inputs_pre['open_acc'] >= 31), 1, 0)

'pub_rec'

In [ ]:
df_inputs_pre['pub_rec'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'pub_rec', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'pub_rec:0',  'pub_rec:1-2',  'pub_rec:3-6', 'pub_rec:>6'

df_inputs_pre['pub_rec:0'] = np.where((df_inputs_pre['pub_rec'] == 0), 1, 0)

df_inputs_pre['pub_rec:1-2'] = np.where((df_inputs_pre['pub_rec'] >= 1) & (df_inputs_pre['pub_rec'] <= 2), 1, 0)

df_inputs_pre['pub_rec:3-6'] = np.where((df_inputs_pre['pub_rec'] >= 3) & (df_inputs_pre['pub_rec'] <= 6), 1, 0)

df_inputs_pre['pub_rec:>6'] = np.where((df_inputs_pre['pub_rec'] > 6), 1, 0)

'total_acc'

In [ ]:
df_inputs_pre['total_acc'].describe()
df_inputs_pre['total_acc'].unique()

In [ ]:
df_inputs_pre['total_acc_factor'] = pd.cut(df_inputs_pre['total_acc'], 50)
df_temp = woe_ordered_continuous(df_inputs_pre, 'total_acc_factor', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'total_acc:<=9', 'total_acc:10-21', 'total_acc:22-33', 'total_acc:34-51', 'total_acc:>51'

df_inputs_pre['total_acc:<=9'] = np.where((df_inputs_pre['total_acc'] <= 9), 1, 0)

df_inputs_pre['total_acc:10-21'] = np.where((df_inputs_pre['total_acc'] >= 10) & (df_inputs_pre['total_acc'] <= 21), 1, 0)

df_inputs_pre['total_acc:22-33'] = np.where((df_inputs_pre['total_acc'] >= 22) & (df_inputs_pre['total_acc'] <= 33), 1, 0)

df_inputs_pre['total_acc:34-51'] = np.where((df_inputs_pre['total_acc'] >= 34) & (df_inputs_pre['total_acc'] <= 51), 1, 0)

df_inputs_pre['total_acc:>51'] = np.where((df_inputs_pre['total_acc'] > 51), 1, 0)

'acc_now_delinq'

In [ ]:
df_inputs_pre['acc_now_delinq'].unique()

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'acc_now_delinq', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
# 'acc_now_delinq:0','acc_now_delinq:>=1'

df_inputs_pre['acc_now_delinq:0'] = np.where((df_inputs_pre['acc_now_delinq'] == 0), 1, 0)

df_inputs_pre['acc_now_delinq:>=1'] = np.where((df_inputs_pre['acc_now_delinq'] >= 1), 1, 0)

'total_rev_hi_lim'

In [ ]:
df_inputs_pre['total_rev_hi_lim'].nunique()

In [ ]:
df_inputs_pre['total_rev_hi_lim'].describe()

In [ ]:
df_inputs_pre_temp = df_inputs_pre[df_inputs_pre['total_rev_hi_lim'] < 200000]

In [ ]:
df_inputs_pre_temp['total_rev_hi_lim_factor'] = pd.cut(df_inputs_pre_temp['total_rev_hi_lim'], 50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre_temp, 'total_rev_hi_lim_factor', df_targets_pre[df_inputs_pre_temp.index])
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'total_rev_hi_lim:<36K', 'total_rev_hi_lim:36K-56K', 'total_rev_hi_lim:56K-92K', 'total_rev_hi_lim:>92K'

df_inputs_pre['total_rev_hi_lim:<36K'] = np.where((df_inputs_pre['total_rev_hi_lim'] < 36000), 1, 0)

df_inputs_pre['total_rev_hi_lim:36K-56K'] = np.where((df_inputs_pre['total_rev_hi_lim'] >= 36000) & (df_inputs_pre['total_rev_hi_lim'] < 56000), 1, 0)

df_inputs_pre['total_rev_hi_lim:56K-92K'] = np.where((df_inputs_pre['total_rev_hi_lim'] >= 56000) & (df_inputs_pre['total_rev_hi_lim'] < 92000), 1, 0)

df_inputs_pre['total_rev_hi_lim:>92K'] = np.where((df_inputs_pre['total_rev_hi_lim'] <= 92000), 1, 0)

'installment'

In [ ]:
df_inputs_pre['installment'].nunique()

In [ ]:
df_inputs_pre['installment_factor'] = pd.cut(df_inputs_pre['installment'],50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'installment_factor', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# do not use 'installment'

'annual_inc'

In [ ]:
df_inputs_pre['annual_inc_factor'] = pd.cut(df_inputs_pre['annual_inc'],50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'annual_inc_factor', df_targets_pre)

In [ ]:
df_temp

In [ ]:
# one dummy for high income 'annual_inc:>140000'

In [ ]:
df_inputs_pre_temp = df_inputs_pre[df_inputs_pre['annual_inc']<=140000]

In [ ]:
df_inputs_pre_temp['annual_inc_factor'] = pd.cut(df_inputs_pre_temp['annual_inc'],50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre_temp, 'annual_inc_factor', df_targets_pre[df_inputs_pre['annual_inc']<=140000])

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
# 'annual_inc:<20K', 'annual_inc:20K-30K', 'annual_inc:30K-40K', 'annual_inc:40K-50K', 'annual_inc:50K-60K', 'annual_inc:60K-70K', 'annual_inc:70K-80K', 'annual_inc:80K-90K', 'annual_inc:90K-100K', 'annual_inc:100K-120K', 'annual_inc:120K-140K', 'annual_inc:>140K'
df_inputs_pre['annual_inc:<20K'] = np.where((df_inputs_pre['annual_inc'] < 20000), 1, 0) 
df_inputs_pre['annual_inc:20K-30K'] = np.where((df_inputs_pre['annual_inc'] < 20000) & (df_inputs_pre['annual_inc'] >= 30000), 1, 0) 
df_inputs_pre['annual_inc:30K-40K'] = np.where((df_inputs_pre['annual_inc'] < 30000) & (df_inputs_pre['annual_inc'] >= 40000), 1, 0) 
df_inputs_pre['annual_inc:40K-50K'] = np.where((df_inputs_pre['annual_inc'] < 40000) & (df_inputs_pre['annual_inc'] >= 50000), 1, 0) 
df_inputs_pre['annual_inc:50K-60K'] = np.where((df_inputs_pre['annual_inc'] < 50000) & (df_inputs_pre['annual_inc'] >= 60000), 1, 0) 
df_inputs_pre['annual_inc:60K-70K'] = np.where((df_inputs_pre['annual_inc'] < 60000) & (df_inputs_pre['annual_inc'] >= 70000), 1, 0) 
df_inputs_pre['annual_inc:70K-80K'] = np.where((df_inputs_pre['annual_inc'] < 70000) & (df_inputs_pre['annual_inc'] >= 80000), 1, 0) 
df_inputs_pre['annual_inc:80K-90K'] = np.where((df_inputs_pre['annual_inc'] < 80000) & (df_inputs_pre['annual_inc'] >= 90000), 1, 0) 
df_inputs_pre['annual_inc:90K-100K'] = np.where((df_inputs_pre['annual_inc'] < 90000) & (df_inputs_pre['annual_inc'] >= 100000), 1, 0) 
df_inputs_pre['annual_inc:100K-120K'] = np.where((df_inputs_pre['annual_inc'] < 100000) & (df_inputs_pre['annual_inc'] >= 120000), 1, 0) 
df_inputs_pre['annual_inc:120K-140K'] = np.where((df_inputs_pre['annual_inc'] < 120000) & (df_inputs_pre['annual_inc'] >= 140000), 1, 0) 
df_inputs_pre['annual_inc:>140K'] = np.where((df_inputs_pre['annual_inc'] >= 140000), 1, 0) 

'mths_since_last_delinq'

In [ ]:
df_inputs_pre_temp = df_inputs_pre[pd.notnull(df_inputs_pre['mths_since_last_delinq'])]

In [ ]:
df_inputs_pre_temp.shape

In [ ]:
df_inputs_pre.shape

In [ ]:
df_inputs_pre_temp['mths_since_last_delinq_factor'] = pd.cut(df_inputs_pre_temp['mths_since_last_delinq'],50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre_temp, 'mths_since_last_delinq_factor', df_targets_pre[df_inputs_pre_temp.index])

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'mths_since_last_delinq:Missing', 'mths_since_last_delinq:0-3', 'mths_since_last_delinq:4-30', 'mths_since_last_delinq:31-56', 'mths_since_last_delinq:>=57'
df_inputs_pre['mths_since_last_delinq:Missing'] = np.where((df_inputs_pre['mths_since_last_delinq'].isnull()), 1, 0)
df_inputs_pre['mths_since_last_delinq:0-3'] = np.where((df_inputs_pre['mths_since_last_delinq'] >= 0) & (df_inputs_pre['mths_since_last_delinq'] < 3), 1, 0)
df_inputs_pre['mths_since_last_delinq:4-30'] = np.where((df_inputs_pre['mths_since_last_delinq'] >= 4) & (df_inputs_pre['mths_since_last_delinq'] < 30), 1, 0)
df_inputs_pre['mths_since_last_delinq:31-56'] = np.where((df_inputs_pre['mths_since_last_delinq'] >= 31) & (df_inputs_pre['mths_since_last_delinq'] < 56), 1, 0)
df_inputs_pre['mths_since_last_delinq:>=57'] = np.where((df_inputs_pre['mths_since_last_delinq'] >= 57), 1, 0)

'dti'

In [ ]:
df_inputs_pre['dti'].nunique()

In [ ]:
df_inputs_pre['dti_factor'] = pd.cut(df_inputs_pre['dti'], 50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'dti_factor', df_targets_pre)
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'dti:<4', 'dti:4-8', 'dti:8-12', 'dti:12-13.6', 'dti:13.6-16.8', 'dti:16.8-20.8', 'dti:20.8-24', 'dti:24-28', 'dti:>28'

df_inputs_pre['dti:<4'] = np.where((df_inputs_pre['dti'] < 4), 1, 0)

df_inputs_pre['dti:4-8'] = np.where((df_inputs_pre['dti'] >= 4) & (df_inputs_pre['dti'] < 8), 1, 0)

df_inputs_pre['dti:8-12'] = np.where((df_inputs_pre['dti'] >= 8) & (df_inputs_pre['dti'] < 12), 1, 0)

df_inputs_pre['dti:12-13.6'] = np.where((df_inputs_pre['dti'] >= 12) & (df_inputs_pre['dti'] < 13.6), 1, 0)

df_inputs_pre['dti:13.6-16.8'] = np.where((df_inputs_pre['dti'] >= 13.6) & (df_inputs_pre['dti'] < 16.8), 1, 0)

df_inputs_pre['dti:16.8-20.8'] = np.where((df_inputs_pre['dti'] >= 16.8) & (df_inputs_pre['dti'] < 20.8), 1, 0)

df_inputs_pre['dti:20.8-24'] = np.where((df_inputs_pre['dti'] >= 20.8) & (df_inputs_pre['dti'] < 24), 1, 0)

df_inputs_pre['dti:24-28'] = np.where((df_inputs_pre['dti'] >= 24) & (df_inputs_pre['dti'] < 28), 1, 0)

df_inputs_pre['dti:>28'] = np.where((df_inputs_pre['dti'] >= 28), 1, 0)

'mths_since_last_record'

In [ ]:
df_inputs_pre['mths_since_last_record'].describe()

In [ ]:
df_inputs_pre['mths_since_last_record'].isnull().sum()

In [ ]:
X_train.shape

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
df_inputs_pre['mths_since_last_record_factor'] = pd.cut(df_inputs_pre['mths_since_last_record'], 50)

In [ ]:
df_temp = woe_ordered_continuous(df_inputs_pre, 'mths_since_last_record_factor', df_targets_pre)

In [ ]:
df_temp

In [ ]:
plot_by_woe(df_temp,90)

In [ ]:
# 'mths_since_last_record:Missing','mths_since_last_record:0-4.8','mths_since_last_record:4.8-24.2','mths_since_last_record:24.2-36.3','mths_since_last_record:36.3-50.8','mths_since_last_record:50.8-67.8','mths_since_last_record:67.8-80','mths_since_last_record:80-84.7', 'mths_since_last_record:>84.7'

df_inputs_pre['mths_since_last_record:Missing'] = np.where((df_inputs_pre['mths_since_last_record'].isnull()), 1, 0)

df_inputs_pre['mths_since_last_record:0-4.8'] = np.where((df_inputs_pre['mths_since_last_record'] < 4.8), 1, 0)

df_inputs_pre['mths_since_last_record:4.8-24.2'] = np.where((df_inputs_pre['mths_since_last_record'] >= 4.8) & (df_inputs_pre['mths_since_last_record'] < 24.2), 1, 0)

df_inputs_pre['mths_since_last_record:24.2-36.3'] = np.where((df_inputs_pre['mths_since_last_record'] >= 24.2) & (df_inputs_pre['mths_since_last_record'] < 36.3), 1, 0)

df_inputs_pre['mths_since_last_record:36.3-50.8'] = np.where((df_inputs_pre['mths_since_last_record'] >= 36.3) & (df_inputs_pre['mths_since_last_record'] < 50.8), 1, 0)

df_inputs_pre['mths_since_last_record:50.8-67.8'] = np.where((df_inputs_pre['mths_since_last_record'] >= 50.8) & (df_inputs_pre['mths_since_last_record'] < 67.8), 1, 0)

df_inputs_pre['mths_since_last_record:67.8-80'] = np.where((df_inputs_pre['mths_since_last_record'] >= 67.8) & (df_inputs_pre['mths_since_last_record'] < 80), 1, 0)

df_inputs_pre['mths_since_last_record:80-84.7'] = np.where((df_inputs_pre['mths_since_last_record'] >= 80) & (df_inputs_pre['mths_since_last_record'] < 84.7), 1, 0)

df_inputs_pre['mths_since_last_record:>84.7'] = np.where((df_inputs_pre['mths_since_last_record'] >= 84.7), 1, 0)

In [ ]:
df_inputs_pre.shape

### save precessed data to X_train

In [ ]:
inputs_train = df_inputs_pre

## Preprocessing the Test Dataset

change from
df_inputs_pre = X_train
df_targets_pre = y_train

to
df_inputs_pre = Xtest
df_targets_pre = y_test

rerun the above section

In [ ]:
inputs_test = df_inputs_pre

In [ ]:
inputs_train.to_csv('loan_data_inputs_train.csv')
y_train.to_csv('loan_data_targets_train.csv')
inputs_test.to_csv('loan_data_inputs_test.csv')
y_test.to_csv('loan_data_targets_test.csv')

# modeling

In [4]:
inputs_train = pd.read_csv('loan_data_inputs_train.csv', index_col=0)
targets_train = pd.read_csv('loan_data_targets_train.csv', index_col=0)
# , header=None)
inputs_test = pd.read_csv('loan_data_inputs_test.csv', index_col=0)
targets_test = pd.read_csv('loan_data_targets_test.csv', index_col=0) 
#, header=None)

C:\Users\YizhiZhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
inputs_train.head()

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,1,0,0,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,0,0,0,1,0,0,0,0,0,0
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,0,0,1,0,0,0,0,0,0
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,0,0,0,1,0,0,0,0,0,0
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,0,1,0,1,0,0,0,0,0,0


In [7]:
inputs_train.shape

(373028, 324)

In [37]:
targets_train.head(10)

,good_bad
427211,1
206088,1
136020,1
412305,0
36159,0
260400,1
29626,1
117003,1
378696,1
159555,1


In [38]:
type(targets_train)

pandas.core.frame.DataFrame

In [14]:
targets_train.shape

(373028, 1)

In [15]:
inputs_test.shape

(93257, 324)

In [16]:
targets_test = pd.read_csv('loan_data_targets_test.csv', index_col=0) 

In [17]:
targets_test.shape

(93257, 1)

In [23]:
inputs_train_ref = inputs_train.loc[:, ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'delinq_2yrs:0',
'delinq_2yrs:1-3',
'delinq_2yrs:>=4',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'open_acc:0',
'open_acc:1-3',
'open_acc:4-12',
'open_acc:13-17',
'open_acc:18-22',
'open_acc:23-25',
'open_acc:26-30',
'open_acc:>=31',
'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',
'total_acc:<=27',
'total_acc:28-51',
'total_acc:>=52',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86'
]]

In [24]:
ref_catogoies = [  'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'delinq_2yrs:>=4',
'inq_last_6mths:>6',
'open_acc:0',
'pub_rec:0-2',
'total_acc:<=27',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [25]:
X_train = inputs_train_ref.drop(ref_catogoies, axis=1)

In [26]:
X_train.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
206088,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
136020,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
412305,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
36159,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [27]:
X_train.shape

(373028, 104)

In [42]:
y_train = targets_train.values.ravel()

In [43]:
y_train.shape

(373028,)

## fit the model

In [28]:
reg = LogisticRegression()

In [44]:
reg.fit(X_train, y_train)

C:\Users\YizhiZhang\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [ ]:
reg.intercept_

In [ ]:
reg.coef_

In [45]:
feature_name = X_train.columns.values

In [46]:
summary_table = pd.DataFrame(columns=['Feature Name'], data= feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table.head()

# Build a model p-values

In [47]:
from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    def __init__(self, *args, **kwargs):
        self.model = linear_model.LogisticRegression(*args, **kwargs)
        
    def fit(self, X, y):
        self.model.fit(X, y)
        demon = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        demon = np.tile(demon, (X.shape[1],1)).T
        F_ij = np.dot((X / demon).T, X)
        Cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [48]:
reg = LogisticRegression_with_p_values()

In [49]:
reg.fit(X_train, y_train)

C:\Users\YizhiZhang\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
